<a href="https://colab.research.google.com/github/flow4u/public/blob/master/COVID_19_Nederland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
IMPORTANT: inital run might fail, just wait a few seconds and do RUN ALL again

Animated gif is created in folder to the right: COVID19_Netherland.gif

'''

try:
  import geopandas as gpd
except:
  !pip install geopandas
  import geopandas as gpd

try: 
  import cbsodata
except:
  !pip install cbsodata
  import cbsodata
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import glob
 

# Zoek op welke data beschikbaar is
metadata = pd.DataFrame(cbsodata.get_meta('84583NED', 'DataProperties'))


In [0]:
# Haal de kaart met gemeentegrenzen op van PDOK
geodata_url = 'https://geodata.nationaalgeoregister.nl/cbsgebiedsindelingen/wfs?request=GetFeature&service=WFS&version=2.0.0&typeName=cbs_gemeente_2019_gegeneraliseerd&outputFormat=json'
gemeentegrenzen = gpd.read_file(geodata_url)

In [0]:
# https://docs.google.com/spreadsheets/d/1Qn740kNVKrKf1-Xs1rMemcXgSCIVo9D-ezC1egrlJd0/edit#gid=1628578164
covid19_nl_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSVeWQqQEvMoFBiDN7Xv-BVriY6Lmtai-FGBQ4fR8Lb_RD5K1aFwnyfjkFrQnuQLOOikfEFQX-bEImZ/pub?gid=2109076169&single=true&output=csv'
covid19_nl = pd.read_csv(covid19_nl_url)
# print(covid19_nl.head())
dates = [h for h in covid19_nl if h[0:4]=='2020']

In [0]:
# Download geboortecijfers en verwijder spaties uit regiocodes
data = pd.DataFrame(cbsodata.get_data('84583NED', select = ['WijkenEnBuurten', 'Codering_3', 'AantalInwoners_5']))
data['Codering_3'] = data['Codering_3'].str.strip()

In [0]:
# Merge
data = pd.merge(data, covid19_nl, left_on='WijkenEnBuurten', right_on='WijkenEnBuurten')

In [0]:
images = []
%matplotlib inline
plt.style.use('ggplot') #'classic')
data_temp=[]
data_temp = data.copy()

for d in dates:
  # data_temp['Per1000'] = data_temp[d]/data_temp['AantalInwoners_5']*1000
  data_temp[d] = data_temp[d]/data_temp['AantalInwoners_5']*1000
  # print(data_temp)
  # print(result.head())

data_temp = data_temp[data_temp['Codering_3'].str.contains("GM")]

# Koppel CBS-data aan geodata met regiocodes
gemeentegrenzen = pd.merge(gemeentegrenzen, data_temp,
                          left_on = "statcode", 
                          right_on = "Codering_3")
# print(gemeentegrenzen.head())

In [0]:
import matplotlib.image as mpimg
import os


cwd = os.getcwd()
png_target = cwd+'/pngs'
try:
  os.makedirs(png_target)
except:
  pass
# print(cwd)

for d in dates:
  # Maak een thematische kaart
  p = gemeentegrenzen.plot(column=d, 
                          figsize = (10,8),
                          legend=True,
                          vmin=0,
                          vmax=10,
                          cmap='rainbow'
                          )
  p.axis('off')
  p.set_title('geregistreerde COVID-19 patiënten per 1000 inwoners op ' + d[6:8]+'-'+d[4:6])
  fig1 = plt.gcf()
  fig1.savefig(png_target + '/' + d+'.png')
  plt.close()


In [0]:
import imageio

png_dir = png_target
images = []
for file_name in sorted(os.listdir(png_dir)):
  # print(file_name)
  if file_name.endswith('.png'):
      file_path = os.path.join(png_dir, file_name)
      images.append(imageio.imread(file_path))
for i in range(0,5):
  images.append(imageio.imread(file_path))
imageio.mimsave('../content/COVID19_Nederland.gif', images, fps=3, loop=0)